In [4]:
import numpy as np
import pandas as pd 
import os
import spacy
import matplotlib.pyplot as plt
import docx

In [5]:
df = pd.read_json("json_training_labelsupdated.json")
training_data = []
for i, one_ent in enumerate(df.annotations.values):
    lis_record =[]
    entities = {}
    for one_ent_ in one_ent[0]['result']:
        lis_single = []
        lis_single.append(one_ent_['value']['start'])
        lis_single.append(one_ent_['value']['end'])
        lis_single.append(one_ent_['value']['labels'][0])
        lis_record.append(tuple(lis_single))
        entities['entities']=lis_record
    training_data.append((list(df.data[i].values())[0], entities))

In [16]:
from spacy.training import Example
import random

TRAIN_DATA=training_data
#load model
nlp = spacy.blank('en')
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
#adding labels
for _, labels in TRAIN_DATA:
     for ent in labels.get('entities'):
        ner.add_label(ent[2])
#adding task to piplines
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
count=0
loss_val = []
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for i in range(100):
        print("iteration :" ,i," Start")
        random.shuffle(TRAIN_DATA)
        losses = {}
        #start training
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            try:
                example = Example.from_dict(doc, annotations)
                nlp.update(
                [example], 
                drop=0.1, 
                sgd=optimizer,
                losses=losses)
            except:
                continue

        loss_val.append(list(losses.values())[0])
        print("Loss in iteration :",i,"loss : ",list(losses.values())[0])

iteration : 0  Start


/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "TENANCY AGREEMENT



This Tenancy Agreement is mad..." with entities "[(86, 96, 'AGREEMENTDATE'), (101, 119, 'PARTYONE')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "RENEWAL OF RENTAL AGREEMENT

This AGREEMENT of Ren..." with entities "[(96, 111, 'AGREEMENTDATE'), (132, 144, 'PARTYONE'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.p

/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "#









































RENTAL ..." with entities "[(134, 158, 'AGREEMENTDATE'), (176, 196, 'PARTYONE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "RENTAL AGREEMENT

This Rental Agreement made and e..." with entities "[(135, 157, 'PARTYONE'), (470, 494, 'PARTYTWO'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.p

Loss in iteration : 0 loss :  5731.781672739999
iteration : 1  Start
Loss in iteration : 1 loss :  1721.2608130155188
iteration : 2  Start
Loss in iteration : 2 loss :  1641.2123042357532
iteration : 3  Start
Loss in iteration : 3 loss :  692.5035041189906
iteration : 4  Start
Loss in iteration : 4 loss :  1772.2201906487144
iteration : 5  Start
Loss in iteration : 5 loss :  828.4776637439476
iteration : 6  Start
Loss in iteration : 6 loss :  622.2507011346912
iteration : 7  Start
Loss in iteration : 7 loss :  785.227080192469
iteration : 8  Start
Loss in iteration : 8 loss :  459.77128782953184
iteration : 9  Start
Loss in iteration : 9 loss :  351.569618714687
iteration : 10  Start
Loss in iteration : 10 loss :  615.1014311828519
iteration : 11  Start
Loss in iteration : 11 loss :  457.49414338203496
iteration : 12  Start
Loss in iteration : 12 loss :  337.8602470888351
iteration : 13  Start
Loss in iteration : 13 loss :  222.1674590308072
iteration : 14  Start
Loss in iteration : 14

In [17]:
# training data i=100,drop =0.1
for text, _ in training_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Jan 10, 2011', 'AGREEMENTDATE'), ('Namashivayam', 'PARTYONE'), ('Mrs.Thenmalar', 'PARTYTWO'), ('14500', 'AGREEMENTVALUE'), ('twelve months', 'TERM'), ('Jan 10, 2011', 'STARTDATE'), ('three months', 'NOTICE')]
Entities [('1stDAY OF SEPTEMBER\n\n2018 (TWO THOUSAND EIGHTEEN)', 'AGREEMENTDATE'), ('SMT PARMESHWARI DEVI JOSHI', 'PARTYONE'), ('TECHNOCULTURE BUILDING CENTRE PRIVATE LIMITED', 'PARTYTWO'), ('36 (Thirty Six) months', 'TERM'), ('36 (Thirty Six) months', 'TERM'), ('36 (Thirty Six) months', 'TERM'), ('1st September 2018', 'STARTDATE'), ('three clear months', 'NOTICE')]
Entities [('15th day of February 2012 (15.02.2012)', 'AGREEMENTDATE'), ('Mr M.SANTOSH', 'PARTYONE'), ('Mr. Jeyanth B', 'PARTYTWO'), ('11 (Eleven) months', 'TERM'), ('11th day of March 2012', 'STARTDATE'), ('10th day of February 2013', 'ENDDATE'), ('2 months', 'NOTICE')]
Entities [('Two thousand fourteen April twenty first (21/04/14),', 'AGREEMENTDATE'), ('S T Nagaveni', 'PARTYONE'), ('Mr Madhu H N', 'PARTY

In [8]:
# training data i=500,drop =0.2
for text, _ in training_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('19th day of July, 2011', 'AGREEMENTDATE'), ('(MARITES M. ALICANTE)', 'PARTYONE'), ('JASMIN B. BAKER and EDWARD RAY BAKER', 'PARTYTWO'), ('THREE (3) MONTHS', 'AGREEMENTVALUE'), ('16 July 2011', 'STARTDATE'), ('16 October 2011', 'ENDDATE'), ('22,000.00', 'AGREEMENTVALUE')]
Entities [('11 day of December, 2008', 'AGREEMENTDATE'), ('PROSPER 4 LESS', 'PARTYONE'), ('700.00', 'AGREEMENTVALUE'), ('15 day of DECEMBER, 2008', 'STARTDATE')]
Entities [('1st May 2005', 'AGREEMENTDATE'), ('Mrs. S.Sakunthala', 'PARTYONE'), ('V.V.Ravi Kian', 'PARTYTWO'), ('11 (Eleven) months', 'TERM'), ('1st April 2010', 'STARTDATE'), ('one month', 'NOTICE')]
Entities [('14th day of July 2010', 'AGREEMENTDATE'), ('Mr. P. JohnsonRavikumar', 'PARTYONE'), ('Mr. Saravanan BV', 'PARTYTWO'), ('11 months', 'TERM'), ('01.04.2010', 'STARTDATE'), ('30.03.2011', 'ENDDATE'), ('three months', 'NOTICE')]
Entities [('19th day of August 2013', 'AGREEMENTDATE'), ('Mrs. M. Geetha,', 'PARTYONE'), ('M/s. Siruvani Traders Priv

In [18]:
# validation data after i=100 and drop = 0.1
import docx2txt
for i in os.listdir('Validation_Data/'):
    text = docx2txt.process('Validation_Data/'+i)
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('1st day of April 2008 (1-04-08)', 'AGREEMENTDATE'), ('Sri Hanumaiah', 'PARTYONE'), ('Sri Vishal Bhardwaj', 'PARTYTWO'), ('1st April 2008', 'STARTDATE'), ('12 months', 'TERM'), ('two months', 'NOTICE')]
Entities [('Mr. KAPIL MEHROTRA', 'PARTYTWO'), ('Mr.B.Kishore', 'PARTYTWO'), ('11 months', 'TERM')]
Entities [('06th day of March 2013', 'PARTYTWO'), ('Smt C.BHAGYAMMA', 'PARTYONE'), ('M/S. JP INTERIO', 'PARTYTWO'), ('11 months', 'TERM'), ('06th day of April 2013', 'STARTDATE'), ('ONE month', 'NOTICE')]
Entities [('first day of September 2011 (01-09-2011)', 'AGREEMENTDATE'), ('Mr. S Parthasarathy', 'PARTYONE'), ('Mr. Hari Kiran Tholeti', 'PARTYTWO'), ('1st September 2011', 'STARTDATE'), ('11 (eleven) months', 'TERM')]
Entities [('17th day of July two thousand', 'AGREEMENTDATE'), ('Mr. SIMON PA', 'PARTYONE'), ('Mr. VAIRAVAN A,', 'PARTYONE'), ('11 (Eleven) months', 'TERM'), ('07-072014', 'STARTDATE'), ('3 (Three) months', 'NOTICE')]
Entities [('1st May 2005', 'AGREEMENTDATE'), (

In [19]:
#training scores i=100, drop =0.1
from spacy.scorer import Scorer
examples = []

for input_, annotations in training_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue

scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9870689655172413, 'ents_r': 1.0, 'ents_f': 0.9934924078091106, 'ents_per_type': {'AGREEMENTDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'PARTYONE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'PARTYTWO': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'AGREEMENTVALUE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TERM': {'p': 0.90625, 'r': 1.0, 'f': 0.9508196721311475}, 'STARTDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'NOTICE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'ENDDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}}}


In [11]:
#training scores i=500, drop =0.2
from spacy.scorer import Scorer
examples = []

for input_, annotations in training_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue

scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9956331877729258, 'ents_r': 0.9956331877729258, 'ents_f': 0.9956331877729258, 'ents_per_type': {'AGREEMENTDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'PARTYONE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'PARTYTWO': {'p': 0.975, 'r': 1.0, 'f': 0.9873417721518987}, 'AGREEMENTVALUE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'STARTDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'ENDDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TERM': {'p': 1.0, 'r': 0.9655172413793104, 'f': 0.9824561403508771}, 'NOTICE': {'p': 1.0, 'r': 1.0, 'f': 1.0}}}


In [14]:
df_v = pd.read_json("json_validation_labelsupdated.json")
validation_data = []
for i, one_ent in enumerate(df_v.annotations.values):
    lis_record =[]
    entities = {}
    for one_ent_ in one_ent[0]['result']:
        lis_single = []
        lis_single.append(one_ent_['value']['start'])
        lis_single.append(one_ent_['value']['end'])
        lis_single.append(one_ent_['value']['labels'][0])
        lis_record.append(tuple(lis_single))
        entities['entities']=lis_record
    validation_data.append((list(df_v.data[i].values())[0], entities))

In [20]:
#validation scores for i=100, drop =0.1
from spacy.scorer import Scorer
examples = []
for input_, annotations in validation_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue
            
scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.8, 'ents_r': 0.7346938775510204, 'ents_f': 0.7659574468085106, 'ents_per_type': {'AGREEMENTDATE': {'p': 0.8, 'r': 0.5714285714285714, 'f': 0.6666666666666666}, 'PARTYONE': {'p': 0.8571428571428571, 'r': 0.75, 'f': 0.7999999999999999}, 'PARTYTWO': {'p': 0.7, 'r': 0.875, 'f': 0.7777777777777777}, 'AGREEMENTVALUE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'TERM': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'STARTDATE': {'p': 0.625, 'r': 0.7142857142857143, 'f': 0.6666666666666666}, 'NOTICE': {'p': 1.0, 'r': 0.8571428571428571, 'f': 0.923076923076923}, 'ENDDATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}


In [ ]:
#validation scores for i=500, drop =0.2
from spacy.scorer import Scorer
examples = []
for input_, annotations in validation_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue
            
scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

In [ ]:
#validation scores for i=100, drop =0.3
from spacy.scorer import Scorer
examples = []
for input_, annotations in validation_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue
            
scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)